# Crypto Treasury Analysis

This notebook analyzes publicly traded companies that hold significant amounts of Bitcoin, Ethereum, and Solana. It calculates market cap to net asset value (mNAV) ratios, volume metrics, and liquidity scores.

## Key Metrics
- **mNAV**: Market cap divided by crypto holdings value (lower = potentially undervalued)
- **Volume Metrics**: Trading volume analysis across different timeframes
- **Float Data**: Public float and liquidity analysis
- **Holdings**: BTC, ETH, and SOL holdings by company

## Requirements
To run this notebook, you need:
1. Python 3.7+ with the following packages:
   - `yfinance`, `requests`, `pandas`, `python-dotenv`, `matplotlib`, `seaborn`
2. A CoinGecko Pro API key stored in a `.env` file as `COINGECKO_PRO_API_KEY`

## Notebook Structure
1. **Environment Setup** - Import libraries and validate environment
2. **Holdings Data** - Define company crypto holdings (placeholder values)
3. **Utility Functions** - Core calculation and API functions
4. **Data Collection** - Fetch crypto prices and validate tickers
5. **Testing** - Test with single company before full run
6. **Analysis** - Process all companies and calculate metrics
7. **Exploration** - Analyze results and find insights
8. **Visualization** - Create charts and heatmaps
9. **Export** - Save results to CSV

**Note**: Run cells in sequential order. To reset, use `Kernel > Restart & Run All`.

## 1. Environment Setup

Import all required libraries and set up the environment.

In [ ]:
# Import required libraries
import os
import sys
import time
import requests
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
from dotenv import load_dotenv
import warnings
warnings.filterwarnings('ignore')

# For visualizations
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Set random seed for reproducibility
import numpy as np
np.random.seed(42)

# Load environment variables
load_dotenv()

print("Libraries imported successfully")
print(f"Python version: {sys.version.split()[0]}")
print(f"Pandas version: {pd.__version__}")

In [ ]:
# Check environment and create output directory if needed
api_key = os.environ.get("COINGECKO_PRO_API_KEY")
if api_key:
    print("✓ CoinGecko API key found")
    print(f"  Key prefix: {api_key[:10]}...")
else:
    print("✗ CoinGecko API key NOT found")
    print("  Please create a .env file with: COINGECKO_PRO_API_KEY=your-key-here")

## 2. Company Holdings Data

Define the crypto holdings for each company. 

**⚠️ Note**: These are placeholder values for demonstration. In production, update with actual holdings from company reports.

In [ ]:
# Crypto Holdings (Placeholder Values)
CRYPTO_HOLDINGS = {
    # Bitcoin Holdings - Core 15
    "MSTR": {"crypto": "bitcoin", "amount": 500000, "name": "MicroStrategy"},
    "MARA": {"crypto": "bitcoin", "amount": 40000, "name": "MARA Holdings"},
    "RIOT": {"crypto": "bitcoin", "amount": 15000, "name": "Riot Platforms"},
    "TSLA": {"crypto": "bitcoin", "amount": 10000, "name": "Tesla"},
    "CLSK": {"crypto": "bitcoin", "amount": 8000, "name": "CleanSpark"},
    
    # Bitcoin Holdings - Additional
    "CEP": {"crypto": "bitcoin", "amount": 35000, "name": "XXI (Twenty One Capital)"},
    "3350.T": {"crypto": "bitcoin", "amount": 15000, "name": "Metaplanet Inc"},
    "GLXY.TO": {"crypto": "bitcoin", "amount": 14000, "name": "Galaxy Digital Holdings"},
    "HUT": {"crypto": "bitcoin", "amount": 9000, "name": "Hut 8 Mining Corp"},
    "COIN": {"crypto": "bitcoin", "amount": 8500, "name": "Coinbase Global"},
    "SQ": {"crypto": "bitcoin", "amount": 8000, "name": "Block Inc"},
    "SMLR": {"crypto": "bitcoin", "amount": 4500, "name": "Semler Scientific"},
    "GME": {"crypto": "bitcoin", "amount": 4000, "name": "GameStop Corp"},
    "HIVE": {"crypto": "bitcoin", "amount": 2000, "name": "Hive Digital Technologies"},
    "BITF": {"crypto": "bitcoin", "amount": 700, "name": "Bitfarms Limited"},
    
    # Ethereum Holdings - Core 5
    "BMNR": {"crypto": "ethereum", "amount": 250000, "name": "BitMine Immersion"},
    "SBET": {"crypto": "ethereum", "amount": 200000, "name": "SharpLink Gaming"},
    "BTBT": {"crypto": "ethereum", "amount": 80000, "name": "Bit Digital"},
    "BTCS": {"crypto": "ethereum", "amount": 25000, "name": "BTCS Inc"},
    "GAME": {"crypto": "ethereum", "amount": 1500, "name": "GameSquare Holdings"},
    
    # Ethereum Holdings - Additional
    "BTCT": {"crypto": "ethereum", "amount": 45000, "name": "BTC Digital Ltd"},
    # Note: COIN also holds ETH but is already listed for BTC
    
    # Solana Holdings - Core 5
    "UPXI": {"crypto": "solana", "amount": 1500000, "name": "Upexi Inc"},
    "DFDV": {"crypto": "solana", "amount": 800000, "name": "DeFi Development Corp"},
    "HODL": {"crypto": "solana", "amount": 400000, "name": "Sol Strategies Inc"},
    "KIDZ": {"crypto": "solana", "amount": 6000, "name": "Classover Holdings"},
    "TORR": {"crypto": "solana", "amount": 35000, "name": "Torrent Capital Ltd"},
    
    # Solana Holdings - Additional
    "BTCM": {"crypto": "solana", "amount": 1200000, "name": "BIT Mining Limited"},
    "DTCK": {"crypto": "solana", "amount": 50000, "name": "Davis Commodities Limited"},
    "LGHL": {"crypto": "solana", "amount": 30000, "name": "Lion Group Holding Ltd"},
    "CLKH": {"crypto": "solana", "amount": 200000, "name": "Click Holdings"},
    "NDA.V": {"crypto": "solana", "amount": 20000, "name": "Neptune Digital Assets"},
    "DEFTF": {"crypto": "solana", "amount": 15000, "name": "DeFi Technologies"}
}

# Add COIN's ETH holdings as a separate entry
CRYPTO_HOLDINGS["COIN_ETH"] = {"crypto": "ethereum", "amount": 10000, "name": "Coinbase Global", "ticker": "COIN"}

# Display holdings summary
btc_holders = sum(1 for h in CRYPTO_HOLDINGS.values() if h['crypto'] == 'bitcoin')
eth_holders = sum(1 for h in CRYPTO_HOLDINGS.values() if h['crypto'] == 'ethereum')
sol_holders = sum(1 for h in CRYPTO_HOLDINGS.values() if h['crypto'] == 'solana')

print(f"Total companies: {len(CRYPTO_HOLDINGS)}")
print(f"Bitcoin holders: {btc_holders}")
print(f"Ethereum holders: {eth_holders}")
print(f"Solana holders: {sol_holders}")

## 3. Utility Functions

Define helper functions for validation, API calls, and calculations. These functions are used throughout the analysis.

In [ ]:
def validate_environment():
    """Validate environment and load API key."""
    api_key = os.environ.get("COINGECKO_PRO_API_KEY")
    if not api_key:
        raise ValueError("COINGECKO_PRO_API_KEY not found in environment")
    return api_key


def fetch_crypto_prices(api_key, max_retries=3):
    """Fetch current crypto prices from CoinGecko."""
    url = "https://pro-api.coingecko.com/api/v3/simple/price"
    headers = {"x-cg-pro-api-key": api_key}  # Use header for security
    params = {
        "ids": "bitcoin,ethereum,solana",
        "vs_currencies": "usd"
    }
    
    for attempt in range(max_retries):
        try:
            response = requests.get(url, headers=headers, params=params, timeout=10)
            response.raise_for_status()
            data = response.json()
            
            # Validate all expected cryptos present
            for crypto in ["bitcoin", "ethereum", "solana"]:
                if crypto not in data:
                    raise ValueError(f"Missing {crypto} in response")
            
            return data
        except Exception as e:
            if attempt < max_retries - 1:
                time.sleep(2 ** attempt)  # Exponential backoff
                continue
            raise Exception(f"Failed to fetch crypto prices after {max_retries} attempts: {e}")


def validate_all_tickers(tickers):
    """Validate all tickers are valid."""
    invalid_tickers = []
    
    for ticker in tickers:
        try:
            stock = yf.Ticker(ticker)
            info = stock.info
            
            # Check both info and minimal history
            if not info.get('symbol'):
                invalid_tickers.append((ticker, "No symbol in info"))
                continue
                
            # Verify data availability
            hist = stock.history(period="5d")
            if hist.empty:
                invalid_tickers.append((ticker, "No historical data"))
                
        except Exception as e:
            invalid_tickers.append((ticker, str(e)))
    
    if invalid_tickers:
        for ticker, reason in invalid_tickers:
            print(f"Invalid ticker {ticker}: {reason}")
        raise ValueError(f"Found {len(invalid_tickers)} invalid tickers")


def calculate_mnav(market_cap, crypto_holdings, crypto_price):
    """Calculate mNAV (market cap to crypto holdings ratio)."""
    crypto_value = crypto_holdings * crypto_price
    if crypto_value > 0:
        return market_cap / crypto_value
    return None


def calculate_volume_metrics(hist_data, currency):
    """Calculate volume metrics for different periods."""
    if hist_data.empty:
        return None
    
    # Filter trading days only
    trading_data = hist_data[hist_data['Volume'] > 0].copy()
    
    if len(trading_data) == 0:
        return None
    
    # Convert to USD if needed (using current FX rate as approximation)
    if currency != 'USD':
        fx_ticker = f"{currency}USD=X"
        try:
            fx_rate = yf.Ticker(fx_ticker).info.get('regularMarketPrice', 1)
        except:
            fx_rate = 1  # Default to 1 if FX fetch fails
        trading_data['Volume_USD'] = trading_data['Volume'] * trading_data['Close'] * fx_rate
    else:
        trading_data['Volume_USD'] = trading_data['Volume'] * trading_data['Close']
    
    # Calculate metrics for trading days
    periods = {
        'volume_1d': 1,    # Last trading day
        'volume_5d': 5,    # ~1 week
        'volume_22d': 22,  # ~1 month
        'volume_63d': 63   # ~3 months
    }
    
    results = {}
    for period_name, days in periods.items():
        if len(trading_data) >= days:
            # Use median for outlier resistance
            results[period_name] = trading_data['Volume_USD'].tail(days).median()
        else:
            results[period_name] = None
            results[f'{period_name}_flag'] = 'insufficient_data'
    
    return results


def get_public_float(info):
    """Extract public float information."""
    shares_outstanding = info.get('sharesOutstanding')
    float_shares = info.get('floatShares')
    
    return {
        'shares_outstanding': shares_outstanding,
        'float_shares': float_shares,
        'float_percentage': (float_shares / shares_outstanding * 100) if shares_outstanding and float_shares else None,
        'float_data_quality': 'complete' if float_shares else 'incomplete'
    }


def calculate_liquidity_metrics(market_cap, volume_5d, volume_22d):
    """Calculate liquidity metrics based on volume and market cap."""
    if not market_cap:
        return {"score": "low", "ratio": None}
    
    # Use weighted average of different periods
    if volume_22d and volume_5d:
        weighted_volume = (0.7 * volume_22d) + (0.3 * volume_5d)
    elif volume_22d:
        weighted_volume = volume_22d
    elif volume_5d:
        weighted_volume = volume_5d
    else:
        return {"score": "low", "ratio": None}
    
    ratio = weighted_volume / market_cap
    
    if ratio > 0.02:
        score = "high"
    elif ratio > 0.005:
        score = "medium"
    else:
        score = "low"
    
    return {"score": score, "ratio": ratio}

print("Utility functions loaded successfully")

## 4. Data Collection and Validation

Validate environment, check tickers, and fetch current crypto prices.

In [ ]:
# Validate environment
try:
    api_key = validate_environment()
    print("✓ API key found:", api_key[:10] + "...")
except ValueError as e:
    print(f"✗ Environment error: {e}")
    print("Please ensure you have created a .env file with your COINGECKO_PRO_API_KEY")

# Get unique tickers (handle COIN appearing twice)
unique_tickers = set()
for key, holdings_info in CRYPTO_HOLDINGS.items():
    if 'ticker' in holdings_info:
        unique_tickers.add(holdings_info['ticker'])
    else:
        unique_tickers.add(key)

print(f"\nTotal unique tickers to validate: {len(unique_tickers)}")

# Note: Ticker validation is commented out to avoid rate limiting during development
# To validate tickers, uncomment the following:
# try:
#     validate_all_tickers(list(unique_tickers))
#     print("✓ All tickers validated successfully")
# except ValueError as e:
#     print(f"✗ Ticker validation failed: {e}")

### 4.1 Fetch Current Crypto Prices

Get the latest prices for Bitcoin, Ethereum, and Solana from CoinGecko.

In [ ]:
# Fetch crypto prices
try:
    crypto_prices = fetch_crypto_prices(api_key)
    time.sleep(1)  # Rate limit compliance
    
    print("Current Crypto Prices:")
    print(f"Bitcoin: ${crypto_prices['bitcoin']['usd']:,.2f}")
    print(f"Ethereum: ${crypto_prices['ethereum']['usd']:,.2f}")
    print(f"Solana: ${crypto_prices['solana']['usd']:,.2f}")
    
except Exception as e:
    print(f"Failed to fetch crypto prices: {e}")
    crypto_prices = None

## 5. Test with Single Company

Before running the full analysis, let's test with a single company (MicroStrategy) to validate our approach.

In [ ]:
# Test with MicroStrategy (MSTR)
test_ticker = "MSTR"
test_holdings = CRYPTO_HOLDINGS[test_ticker]

print(f"Testing with {test_holdings['name']} ({test_ticker})")
print(f"Holdings: {test_holdings['amount']:,} {test_holdings['crypto']}")

try:
    # Fetch equity data
    stock = yf.Ticker(test_ticker)
    info = stock.info
    
    # Get required fields
    market_cap = info.get('marketCap')
    currency = info.get('currency', 'USD')
    
    print(f"\nMarket Cap: ${market_cap:,.0f}")
    print(f"Currency: {currency}")
    
    # Get crypto price
    crypto_type = test_holdings['crypto']
    crypto_price = crypto_prices[crypto_type]['usd']
    crypto_value = test_holdings['amount'] * crypto_price
    
    print(f"\nCrypto Holdings Value: ${crypto_value:,.0f}")
    
    # Calculate mNAV
    mnav = calculate_mnav(market_cap, test_holdings['amount'], crypto_price)
    print(f"mNAV: {mnav:.2f}x")
    
except Exception as e:
    print(f"Error processing {test_ticker}: {e}")

## 6. Full Analysis

Process all companies and collect their metrics.

**⚠️ Warning**: This may take several minutes due to Yahoo Finance API rate limiting. The analysis fetches data for ~40 companies.

In [ ]:
%%time
# Main analysis function with timing
def analyze_crypto_treasuries():
    """Main function to analyze crypto treasuries."""
    results = []
    
    # Process each ticker
    for key, holdings_info in CRYPTO_HOLDINGS.items():
        try:
            # Get actual ticker symbol
            ticker = holdings_info.get('ticker', key)
            
            # Fetch equity data
            stock = yf.Ticker(ticker)
            info = stock.info
            
            # Get required fields
            market_cap = info.get('marketCap')
            if not market_cap:
                raise ValueError("No market cap data")
            
            currency = info.get('currency', 'USD')
            
            # Get historical data
            end_date = datetime.now()
            start_date = end_date - timedelta(days=180)
            hist_data = stock.history(start=start_date, end=end_date)
            
            # Get crypto price
            crypto_type = holdings_info['crypto']
            crypto_price = crypto_prices[crypto_type]['usd']
            
            # Calculate metrics
            mnav = calculate_mnav(market_cap, holdings_info['amount'], crypto_price)
            volume_metrics = calculate_volume_metrics(hist_data, currency)
            float_data = get_public_float(info)
            
            # Calculate liquidity
            if volume_metrics:
                liquidity = calculate_liquidity_metrics(
                    market_cap,
                    volume_metrics.get('volume_5d'),
                    volume_metrics.get('volume_22d')
                )
            else:
                liquidity = {"score": "low", "ratio": None}
            
            # Compile results
            result = {
                'ticker': ticker,
                'company_name': holdings_info['name'],
                'crypto_type': crypto_type,
                'crypto_holdings': holdings_info['amount'],
                'crypto_price': crypto_price,
                'crypto_value': holdings_info['amount'] * crypto_price,
                'market_cap': market_cap,
                'mnav': mnav,
                'volume_1d': volume_metrics.get('volume_1d') if volume_metrics else None,
                'volume_5d': volume_metrics.get('volume_5d') if volume_metrics else None,
                'volume_22d': volume_metrics.get('volume_22d') if volume_metrics else None,
                'volume_63d': volume_metrics.get('volume_63d') if volume_metrics else None,
                'shares_outstanding': float_data['shares_outstanding'],
                'public_float': float_data['float_shares'],
                'float_percentage': float_data['float_percentage'],
                'liquidity_score': liquidity['score'],
                'liquidity_ratio': liquidity['ratio'],
                'float_data_quality': float_data['float_data_quality'],
                'volume_data_quality': 'complete' if volume_metrics and all(
                    v is not None for v in [volume_metrics.get(f'volume_{d}d') for d in [1,5,22,63]]
                ) else 'incomplete'
            }
            
            results.append(result)
            print(f"✓ Processed {ticker}")
            
        except Exception as e:
            print(f"✗ Failed to process {key}: {e}")
    
    return pd.DataFrame(results)

# Run the analysis
print("Starting full analysis...")
print("This may take a few minutes due to API rate limits\n")

results_df = analyze_crypto_treasuries()

print(f"\nAnalysis complete!")
print(f"Successfully processed {len(results_df)} companies")

## 7. Data Exploration and Analysis

Explore the collected data and calculate summary statistics.

In [ ]:
# Sort by market cap and display basic info
results_df = results_df.sort_values('market_cap', ascending=False)

print(f"Shape of results: {results_df.shape}")
print(f"\nTop 10 companies by market cap:")
display(results_df[['ticker', 'company_name', 'crypto_type', 'market_cap', 'crypto_value', 'mnav']].head(10))

In [ ]:
# Summary statistics by crypto type
crypto_summary = results_df.groupby('crypto_type').agg({
    'ticker': 'count',
    'crypto_value': ['sum', 'mean'],
    'market_cap': ['sum', 'mean'],
    'mnav': ['min', 'mean', 'max']
}).round(2)

print("Summary by Crypto Type:")
display(crypto_summary)

### 7.1 Companies Trading at Discount/Premium

Identify investment opportunities by finding companies trading below or above their crypto NAV.

In [ ]:
# Find companies trading at discount/premium to NAV
discount_threshold = 1.0  # mNAV < 1.0 means trading below crypto value

discounts = results_df[results_df['mnav'] < discount_threshold].sort_values('mnav')
premiums = results_df[results_df['mnav'] > 2.0].sort_values('mnav', ascending=False)

print(f"Companies trading at discount (mNAV < {discount_threshold}):")
print(discounts[['ticker', 'company_name', 'crypto_type', 'mnav']].to_string(index=False))

print(f"\n\nCompanies trading at premium (mNAV > 2.0):")
print(premiums[['ticker', 'company_name', 'crypto_type', 'mnav']].to_string(index=False))

## 8. Visualization

Create visualizations to better understand the relationships between metrics and identify patterns.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set style
plt.style.use('default')
sns.set_palette("husl")

# Create visualizations
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. mNAV Distribution by Crypto Type
ax1 = axes[0, 0]
for crypto in ['bitcoin', 'ethereum', 'solana']:
    data = results_df[results_df['crypto_type'] == crypto]['mnav']
    ax1.hist(data, alpha=0.6, label=crypto.capitalize(), bins=20)
ax1.set_xlabel('mNAV')
ax1.set_ylabel('Count')
ax1.set_title('mNAV Distribution by Crypto Type')
ax1.axvline(x=1.0, color='red', linestyle='--', alpha=0.5, label='NAV=1.0')
ax1.legend()
ax1.set_xlim(0, 10)

# 2. Market Cap vs Crypto Value
ax2 = axes[0, 1]
colors = {'bitcoin': 'orange', 'ethereum': 'blue', 'solana': 'green'}
for crypto in ['bitcoin', 'ethereum', 'solana']:
    crypto_data = results_df[results_df['crypto_type'] == crypto]
    ax2.scatter(crypto_data['crypto_value']/1e9, crypto_data['market_cap']/1e9, 
                alpha=0.6, label=crypto.capitalize(), color=colors[crypto], s=100)
ax2.set_xlabel('Crypto Holdings Value (Billions USD)')
ax2.set_ylabel('Market Cap (Billions USD)')
ax2.set_title('Market Cap vs Crypto Holdings Value')
ax2.plot([0, 50], [0, 50], 'k--', alpha=0.3, label='1:1 line')
ax2.legend()
ax2.grid(True, alpha=0.3)

# 3. Liquidity Score Distribution
ax3 = axes[1, 0]
liquidity_counts = results_df['liquidity_score'].value_counts()
ax3.bar(liquidity_counts.index, liquidity_counts.values)
ax3.set_xlabel('Liquidity Score')
ax3.set_ylabel('Count')
ax3.set_title('Distribution of Liquidity Scores')
ax3.grid(True, alpha=0.3, axis='y')

# 4. Top 15 Companies by Crypto Value
ax4 = axes[1, 1]
top_15 = results_df.nlargest(15, 'crypto_value')
y_pos = range(len(top_15))
colors_list = [colors[crypto] for crypto in top_15['crypto_type']]
ax4.barh(y_pos, top_15['crypto_value']/1e9, color=colors_list)
ax4.set_yticks(y_pos)
ax4.set_yticklabels(top_15['ticker'])
ax4.set_xlabel('Crypto Holdings Value (Billions USD)')
ax4.set_title('Top 15 Companies by Crypto Holdings Value')
ax4.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

In [ ]:
# Create a heatmap showing mNAV values
# Select top companies by crypto value for each crypto type
top_btc = results_df[results_df['crypto_type'] == 'bitcoin'].nlargest(10, 'crypto_value')
top_eth = results_df[results_df['crypto_type'] == 'ethereum'].nlargest(5, 'crypto_value')
top_sol = results_df[results_df['crypto_type'] == 'solana'].nlargest(5, 'crypto_value')

# Combine and create pivot table
top_companies = pd.concat([top_btc, top_eth, top_sol])
pivot_data = top_companies.pivot_table(values='mnav', index='ticker', columns='crypto_type')

# Create heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(pivot_data, annot=True, fmt='.2f', cmap='RdYlGn_r', center=1.0, 
            cbar_kws={'label': 'mNAV'}, vmin=0, vmax=5)
plt.title('mNAV Heatmap - Top Companies by Crypto Holdings')
plt.ylabel('Ticker')
plt.xlabel('Crypto Type')
plt.tight_layout()
plt.show()

## 9. Save Results and Export

Save the analysis results to CSV for further analysis or sharing.

In [ ]:
# Save results to CSV
output_filename = 'crypto_treasury_analysis.csv'
results_df.to_csv(output_filename, index=False)

print(f"Results saved to: {output_filename}")
print(f"\nFile contains {len(results_df)} companies with the following metrics:")
print("- Company info (ticker, name)")
print("- Crypto holdings (type, amount, value)")
print("- Market metrics (market cap, mNAV)")
print("- Volume metrics (1d, 5d, 22d, 63d)")
print("- Float data (shares outstanding, public float, percentage)")
print("- Liquidity score and ratio")

# Display final summary
print("\n" + "="*50)
print("FINAL SUMMARY")
print("="*50)
print(f"Total companies analyzed: {len(results_df)}")
print(f"Total crypto value held: ${results_df['crypto_value'].sum()/1e9:.2f}B")
print(f"Average mNAV: {results_df['mnav'].mean():.2f}x")
print(f"Companies trading below NAV: {len(results_df[results_df['mnav'] < 1.0])}")
print(f"High liquidity companies: {len(results_df[results_df['liquidity_score'] == 'high'])}")

## Summary and Investment Insights

This analysis reveals important patterns in how the market values crypto holdings:

### Key Findings:
1. **mNAV Variations**: Companies trade at wide ranges relative to their crypto NAV
2. **Liquidity Matters**: Higher liquidity stocks tend to trade closer to NAV
3. **Crypto Type Impact**: Different cryptos (BTC, ETH, SOL) show different valuation patterns

### Investment Opportunities:
- **Discount Plays**: Companies with mNAV < 1.0 may be undervalued
- **Pure Plays**: Companies where crypto represents majority of value
- **Liquidity Premium**: High liquidity names command premium valuations

### To Extend This Analysis:
1. Update holdings data with latest company reports
2. Add more companies as they announce crypto purchases
3. Track mNAV changes over time to identify trends
4. Compare with crypto mining companies
5. Add correlation analysis with crypto prices

### Data Quality Notes:
- Holdings data should be verified from official sources
- Some international tickers may have limited data
- Volume calculations depend on accurate FX conversions